# Ablation V0: LoRA Minimal Baseline (r=16, alpha=32)

This notebook is part of an ablation study to compare different fine-tuning approaches.

## Model and Approach
- **Base Model**: `facebook/mms-tts-amh` (VITS architecture, ~36.3M parameters)
- **Fine-tuning Method**: LoRA (r=16, alpha=32)
- **Task**: Text-to-Speech (TTS)
- **Domain**: Legal Amharic text
- **Dataset**: Dataset_4.0h
- **Text Format**: Romanized Amharic (using uroman package)


## 1. Installation and Setup


In [ ]:
# Force single GPU usage to avoid DataParallel issues on Kaggle
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'



In [ ]:
%pip install -q transformers datasets accelerate peft torchaudio librosa soundfile uroman scipy matplotlib


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import librosa
import soundfile as sf
import scipy.io.wavfile
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from transformers import (
    VitsModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType
)

from datasets import Dataset, DatasetDict

# Import uroman for romanization
import uroman

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")



In [ ]:
# Set random seed for reproducibility
SEED = 42 # Change to 123, 456 for different seeds

import random
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


print(f"Random seed set to: {SEED}")


In [ ]:
# Kaggle: dataset is provided via /kaggle/input, no Google Drive mount needed
print('Running on Kaggle - using input dataset in /kaggle/input')


## 2. Configuration


In [ ]:
MODEL_NAME = "facebook/mms-tts-amh"

# Dataset paths - using Dataset_4.0h
AUDIO_DIR = "/kaggle/input/dataset-4-0h/Dataset_4.0h/audio"
TRAIN_CSV = "/kaggle/input/dataset-4-0h/Dataset_4.0h/train.csv"
VAL_CSV = "/kaggle/input/dataset-4-0h/Dataset_4.0h/val.csv"
TEST_CSV = "/kaggle/input/dataset-4-0h/Dataset_4.0h/test.csv"

OUTPUT_DIR = "/kaggle/working/mms_tts_lora_v0"

# LoRA Configuration
LORA_CONFIG = {
 "r": 16,
 "lora_alpha": 32,
 "target_modules": ["q_proj", "v_proj", "k_proj", "out_proj"],
 "lora_dropout": 0.1,
 "bias": "none",
 "task_type": "FEATURE_EXTRACTION"
}

# Training configuration
TRAINING_ARGS = {
 "output_dir": OUTPUT_DIR,
 "per_device_train_batch_size": 8, # Reduced for T4 GPU
 "per_device_eval_batch_size": 4,
 "gradient_accumulation_steps": 8, # Effective batch size 16
 "learning_rate": 5e-6,
 "warmup_steps": 500,
 "max_steps": 3000, # Increased for larger dataset
 "gradient_checkpointing": True,
 "fp16": True,
 "eval_strategy": "steps",
 "eval_steps": 250,
 "save_strategy": "steps",
 "save_steps": 250,
 "save_total_limit": 5,
 "load_best_model_at_end": True,
 "metric_for_best_model": "loss",
 "greater_is_better": False,
 "logging_steps": 50,
 "report_to": "none",
 "push_to_hub": False
}

print("Configuration:")
print(f" Model: {MODEL_NAME}")
print(f" Output directory: {OUTPUT_DIR}")
print(f" LoRA rank (r): {LORA_CONFIG['r']}")
print(f" LoRA alpha: {LORA_CONFIG['lora_alpha']}")



## 3. Load and Prepare Data



In [ ]:
def load_csv_split(csv_path, audio_dir):
    """Load a CSV split and return list of (audio_path, transcription) tuples"""
    df = pd.read_csv(csv_path)

    data = []
    for _, row in df.iterrows():
        audio_path = Path(audio_dir) / row['file_name']
        transcription = str(row['transcription']).strip()

        if audio_path.exists():
            data.append({
                'audio_path': str(audio_path),
                'transcription': transcription
            })
        else:
            print(f"Warning: Audio file not found: {audio_path}")

    return data

train_data = load_csv_split(TRAIN_CSV, AUDIO_DIR)
val_data = load_csv_split(VAL_CSV, AUDIO_DIR)
test_data = load_csv_split(TEST_CSV, AUDIO_DIR)

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nTotal samples: {len(train_data) + len(val_data) + len(test_data)}")



## 4. Romanize Text Using Uroman



In [ ]:
# Initialize uroman instance
try:
    uroman_obj = uroman.Uroman()
    print("Uroman initialized successfully")
except Exception as e:
    print(f"Error initializing uroman: {e}")
    uroman_obj = None

def romanize_text(text):
    """Convert Ge'ez script Amharic text to Romanized format using uroman"""
    if uroman_obj is None:
        return text # Return original if uroman not available

    try:
        # Uroman.romanize_string() is the correct method
        romanized = uroman_obj.romanize_string(text)
        return romanized.strip()
    except Exception as e:
        print(f"Error romanizing text: {text[:50]}... Error: {e}")
        return text # Fallback to original if romanization fails

# Test romanization on a sample
sample_text = train_data[0]['transcription']
print(f"Original (Ge'ez): {sample_text}")
romanized_sample = romanize_text(sample_text)
print(f"Romanized: {romanized_sample}")



In [ ]:
# Romanize all transcriptions
print("Romanizing training data...")
for item in tqdm(train_data, desc="Train"):
 item['transcription_romanized'] = romanize_text(item['transcription'])

print("Romanizing validation data...")
for item in tqdm(val_data, desc="Val"):
 item['transcription_romanized'] = romanize_text(item['transcription'])

print("Romanizing test data...")
for item in tqdm(test_data, desc="Test"):
 item['transcription_romanized'] = romanize_text(item['transcription'])

print("\nRomanization complete!")
print(f"Sample - Original: {train_data[0]['transcription']}")
print(f"Sample - Romanized: {train_data[0]['transcription_romanized']}")


## 5. Load Model and Tokenizer


In [ ]:
print(f"Loading model: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = VitsModel.from_pretrained(MODEL_NAME)

print(f"Model loaded successfully!")
print(f"Sampling rate: {model.config.sampling_rate} Hz")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

if torch.cuda.is_available():
 model = model.to("cuda")
 print("Model moved to CUDA")


## 6. Test Inference (Before Training)

Test the model with romanized text to verify it works correctly.


In [ ]:
def synthesize_speech(model, tokenizer, text_romanized, output_path):
    """Synthesize speech from romanized text"""
    inputs = tokenizer(text_romanized, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        output = model(**inputs).waveform

    # Move tensor to CPU before converting to numpy
    if isinstance(output, torch.Tensor):
        output = output.cpu()

    # Save audio
    scipy.io.wavfile.write(
        output_path,
        rate=model.config.sampling_rate,
        data=output.numpy().T
    )

    return output_path

# Test with sample text (after romanization)
if len(train_data) > 0 and 'transcription_romanized' in train_data[0]:
    test_text_romanized = train_data[0]['transcription_romanized']
    test_output = "test_synthesized_before_training.wav"

    print(f"Testing synthesis with: {test_text_romanized}")
    synthesize_speech(model, tokenizer, test_text_romanized, test_output)
    print(f"Test audio saved to: {test_output}")
else:
    print("Please run romanization cells first!")



## 7. Apply LoRA (if supported)

**Important Note**: VITS models use a different architecture than transformers. LoRA support may be limited. We'll attempt to apply it, but may need to use standard fine-tuning if LoRA is not compatible.


In [ ]:
# Attempt to apply LoRA
# Note: This may fail for VITS models - if so, we'll need to use standard fine-tuning
try:
    # Check available modules in the model
    print("Checking model structure for LoRA-compatible modules...")
    target_modules_found = []
    for name, module in model.named_modules():
        module_type = type(module).__name__
        if any(target in name.lower() for target in ["linear", "proj", "dense"]):
            if "Linear" in module_type or "Conv" in module_type:
                target_modules_found.append(name)

    print(f"Found {len(target_modules_found)} potential target modules")
    if len(target_modules_found) > 0:
        print("Sample modules:", target_modules_found[:5])

    # Try to configure LoRA with found modules
    # VITS models may not have standard transformer modules
    # We may need to identify the correct target modules
    lora_config = LoraConfig(
        r=LORA_CONFIG["r"],
        lora_alpha=LORA_CONFIG["lora_alpha"],
        target_modules=LORA_CONFIG["target_modules"], # Try standard modules first
        lora_dropout=LORA_CONFIG["lora_dropout"],
        bias=LORA_CONFIG["bias"],
        task_type=TaskType.FEATURE_EXTRACTION
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    print("\nLoRA adapters applied successfully!")

except Exception as e:
    print(f"LoRA application failed: {e}")
    print("\nNote: VITS models may not support standard LoRA.")
    print("We may need to use standard fine-tuning instead.")
    print("Continuing with standard model for now...")

# Remove any existing wrapper code (safety check)
# If you have any wrapper code, remove it first before running this

# Check if LoRA was successfully applied
from peft import PeftModel

if isinstance(model, PeftModel):
    print("Model is a PeftModel")
    print(f"Base model type: {type(model.base_model)}")
    print(f"Has base_model.forward: {hasattr(model.base_model, 'forward')}")

    # Try to check if we can access the underlying VITS model
    if hasattr(model.base_model, 'model'):
        print(f"Base model has 'model' attribute: {type(model.base_model.model)}")
else:
    print("Model is NOT a PeftModel - LoRA may not have been applied")

# Fix: Patch PEFT's forward to filter out inputs_embeds (VitsModel doesn't accept it)
# This is necessary because PEFT includes inputs_embeds in its forward signature,
# but VitsModel doesn't accept it
if isinstance(model, PeftModel):
    import functools
    
    # Store the original forward method
    original_peft_forward = model.forward
    
    @functools.wraps(original_peft_forward)
    def patched_peft_forward(*args, **kwargs):
        # Remove inputs_embeds if present (VitsModel doesn't accept it)
        kwargs.pop('inputs_embeds', None)
        # Also remove task_ids which PEFT might pass
        kwargs.pop('task_ids', None)
        # Filter to only valid VitsModel arguments
        valid_kwargs = {k: v for k, v in kwargs.items() 
                        if k in ['input_ids', 'attention_mask', 'speaker_id', 
                                'output_attentions', 'output_hidden_states', 'return_dict', 'labels']}
        # Call the original forward with filtered kwargs
        return original_peft_forward(*args, **valid_kwargs)
    
    # Replace the forward method
    model.forward = patched_peft_forward
    
    # Also patch the base_model's forward to catch any internal calls
    if hasattr(model, 'base_model'):
        # Helper function to create a patched forward
        def create_patched_forward(original_forward):
            @functools.wraps(original_forward)
            def patched_forward(*args, **kwargs):
                kwargs.pop('inputs_embeds', None)
                kwargs.pop('task_ids', None)
                valid_kwargs = {k: v for k, v in kwargs.items() 
                                if k in ['input_ids', 'attention_mask', 'speaker_id', 
                                        'output_attentions', 'output_hidden_states', 'return_dict', 'labels']}
                return original_forward(*args, **valid_kwargs)
            return patched_forward
        
        # Recursively patch nested base_models
        current = model.base_model
        depth = 0
        max_depth = 5 # Safety limit
        
        while depth < max_depth:
            if hasattr(current, 'forward'):
                current.forward = create_patched_forward(current.forward)
            
            # Move to next level
            if hasattr(current, 'base_model'):
                current = current.base_model
            elif hasattr(current, 'model'):
                current = current.model
            else:
                break
            depth += 1
        
    print("Patched PEFT model forward to filter out unsupported arguments (inputs_embeds, etc.)")



## 8. Training Setup

**Important**: VITS models require custom training loops as they don't use the standard HuggingFace Trainer interface. This section provides a basic structure. For actual training, you may need to implement a custom training loop.

**Alternative Approach**: If LoRA doesn't work, consider:
1. Standard fine-tuning (full model fine-tuning)
2. Using a different TTS model that supports LoRA better
3. Implementing custom LoRA for VITS architecture


In [ ]:
# Custom training loop for VITS model
# Note: This is a simplified training loop - VITS training is complex
# You may need to adapt this based on your specific requirements

from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn

# Prepare datasets for training
train_dataset_list = [
 {'text': item['transcription_romanized'], 'audio_path': item['audio_path']}
 for item in train_data
]

val_dataset_list = [
 {'text': item['transcription_romanized'], 'audio_path': item['audio_path']}
 for item in val_data
]

def load_audio(audio_path, target_sr=16000):
 """Load and resample audio"""
 audio, sr = librosa.load(audio_path, sr=target_sr)
 return audio

print(f"Prepared {len(train_dataset_list)} training samples")
print(f"Prepared {len(val_dataset_list)} validation samples")
print("\nNOTE: Full VITS training requires complex loss functions (mel-spectrogram loss, etc.)")
print("This is a simplified structure. For production use, implement full VITS training objectives.")


In [ ]:
# Memory optimization setup before training
import os

# Set PyTorch CUDA memory allocation config to reduce fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    # Print current memory usage
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print("GPU Memory Status:")
    print(f" Allocated: {allocated:.2f} GB")
    print(f" Reserved: {reserved:.2f} GB")
    print(f" Total: {total:.2f} GB")
    print(f" Free: {total - reserved:.2f} GB")
    print()
    
    # Check model memory usage
    if hasattr(model, 'parameters'):
        model_params = sum(p.numel() * p.element_size() for p in model.parameters()) / 1e9
        print(f"Model parameters memory: ~{model_params:.2f} GB (FP32)")
        if TRAINING_ARGS.get("fp16", False):
            print(f" With FP16: ~{model_params / 2:.2f} GB")
        print()



## 9. Train Model

**Note**: This is a simplified training approach. Full VITS training requires complex loss functions. For production, you may need to implement proper VITS training objectives.


In [ ]:
# Initialize tracking lists for training/validation metrics
train_losses = []
val_losses = []
train_steps = []
val_steps = []

print('Initialized metric tracking lists: train_losses, val_losses, train_steps, val_steps')


In [ ]:
# Ensure output directory exists in Kaggle working directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'Output directory: {OUTPUT_DIR}')


In [ ]:
# Ensure model is not wrapped in DataParallel and is on single device (Kaggle)
if isinstance(model, torch.nn.DataParallel):
 print("Unwrapping model from DataParallel...")
 model = model.module

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"Model on device: {device}")
print(f"Visible GPUs: {torch.cuda.device_count()}")


In [ ]:
# Validation evaluation function
def evaluate_validation(model, tokenizer, val_dataset_list, batch_size=4, use_fp16=False):
    """Run validation evaluation on validation dataset."""
    model.eval()
    total_loss = 0.0
    num_samples = 0
 
    with torch.no_grad():
        for i in range(0, len(val_dataset_list), batch_size):
            batch = val_dataset_list[i:i+batch_size]
 
            try:
                # Tokenize batch text
                texts = [item['text'] for item in batch]
                inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
 
                if torch.cuda.is_available():
                    inputs = {k: v.to("cuda") for k, v in inputs.items()}
 
                # Filter out inputs_embeds which VitsModel doesn't accept
                valid_inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
 
                # Forward pass
                if use_fp16:
                    from torch.cuda.amp import autocast
                    with autocast():
                        outputs = model(**valid_inputs)
                else:
                    outputs = model(**valid_inputs)
 
                # Compute loss (simplified - same as training)
                loss = torch.tensor(0.0, device=inputs['input_ids'].device)
 
                total_loss += loss.item() * len(batch)
                num_samples += len(batch)
 
                del outputs, inputs, valid_inputs
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
 
            except Exception as e:
                print(f"Error in validation batch: {e}")
                continue
 
    model.train()
    avg_val_loss = total_loss / num_samples if num_samples > 0 else 0.0
    return avg_val_loss

print("Validation evaluation function defined.")



In [ ]:
# Simplified training loop with MEMORY OPTIMIZATION
# WARNING: This is a basic structure - full VITS training is more complex

# Import required modules
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
import librosa
import numpy as np

# Helper function to compute mel-spectrogram on GPU using PyTorch
def compute_mel_spectrogram_torch(waveform, sample_rate=16000, n_mels=80, n_fft=1024, hop_length=256):
    """Compute mel-spectrogram from waveform using PyTorch (GPU-friendly)"""
    # Ensure waveform is on the right device
    device = waveform.device

    # Convert to mono if needed
    if len(waveform.shape) > 2:
        waveform = waveform.squeeze(1)  # Remove channel dimension if present

    # Use PyTorch's STFT
    window = torch.hann_window(n_fft, device=device)
    stft = torch.stft(
        waveform,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=n_fft,
        window=window,
        center=True,
        normalized=False,
        onesided=True,
        return_complex=True
    )

    # Compute magnitude spectrogram
    magnitude = torch.abs(stft)

    # Create mel filterbank (simplified - you may want to use proper mel scale)
    # For now, use a simple approximation
    mel_basis = torch.ones(n_mels, magnitude.shape[-2], device=device) * (1.0 / magnitude.shape[-2])

    # Apply mel filterbank
    mel_spec = torch.matmul(mel_basis, magnitude)

    # Convert to log scale (add small epsilon to avoid log(0))
    mel_spec_db = torch.log10(mel_spec + 1e-10)

    return mel_spec_db

# VITS training loss function (GPU-optimized)
def vits_training_loss(predicted_waveform, target_waveform, sample_rate=16000):
    """
    Compute training loss for VITS model.
    Uses a combination of waveform L1 loss and mel-spectrogram loss.
    All computations stay on GPU.
    """
    # Ensure same length (truncate)
    min_len = min(predicted_waveform.shape[-1], target_waveform.shape[-1])
    pred = predicted_waveform[..., :min_len]
    target = target_waveform[..., :min_len]

    # Waveform L1 loss
    waveform_loss = F.l1_loss(pred, target)

    # Mel-spectrogram loss (computed on GPU)
    try:
        pred_mel = compute_mel_spectrogram_torch(pred, sample_rate)
        target_mel = compute_mel_spectrogram_torch(target, sample_rate)

        # Ensure same dimensions
        min_time = min(pred_mel.shape[-1], target_mel.shape[-1])
        pred_mel = pred_mel[..., :min_time]
        target_mel = target_mel[..., :min_time]

        mel_loss = F.l1_loss(pred_mel, target_mel)
    except Exception as e:
        # Fallback to waveform loss only if mel computation fails
        print(f"Warning: Mel-spectrogram computation failed: {e}")
        mel_loss = torch.tensor(0.0, device=waveform_loss.device, requires_grad=True)

    # Combined loss (weighted)
    total_loss = waveform_loss + 0.5 * mel_loss

    return total_loss, waveform_loss, mel_loss

# Enable gradient checkpointing to save memory
if TRAINING_ARGS.get("gradient_checkpointing", False):
    try:
        if hasattr(model, 'gradient_checkpointing_enable'):
            model.gradient_checkpointing_enable()
            print(" Gradient checkpointing enabled")
        elif hasattr(model, 'base_model') and hasattr(model.base_model, 'gradient_checkpointing_enable'):
            model.base_model.gradient_checkpointing_enable()
            print(" Gradient checkpointing enabled (via base_model)")
    except Exception as e:
        print(f"Warning: Could not enable gradient checkpointing: {e}")

# Enable FP16 mixed precision training
use_fp16 = TRAINING_ARGS.get("fp16", False)
if use_fp16:
    scaler = GradScaler()
    print(" FP16 mixed precision enabled")
else:
    scaler = None

model.train()
optimizer = AdamW(model.parameters(), lr=TRAINING_ARGS["learning_rate"], weight_decay=0.01)
# Initialize schedulers
warmup_steps = TRAINING_ARGS.get("warmup_steps", 500)
warmup_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=TRAINING_ARGS["max_steps"]
)
plateau_scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=True
)


gradient_accumulation_steps = TRAINING_ARGS["gradient_accumulation_steps"]
max_steps = TRAINING_ARGS["max_steps"]
target_sr = 16000  # VITS typically uses 16kHz

# Create DataLoaders with multiprocessing for efficient audio loading
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,  # Parallel audio loading
    pin_memory=True if torch.cuda.is_available() else False,
    persistent_workers=True,  # Keep workers alive between epochs
    prefetch_factor=2,  # Prefetch batches
    collate_fn=collate_fn  # Use custom collate function for padding
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,  # Fewer workers for validation
    pin_memory=True if torch.cuda.is_available() else False,
    collate_fn=collate_fn  # Use custom collate function for padding
)

print(f"Created DataLoaders with {train_loader.num_workers} workers for training")

print("Starting training...")
print(f"Training steps: {max_steps}")
print(f"Learning rate: {TRAINING_ARGS['learning_rate']}")
print(f"Batch size: {batch_size}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Effective batch size: {batch_size * gradient_accumulation_steps}")
print(f"FP16: {use_fp16}")
print()

# Clear CUDA cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"Initial GPU memory: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print()

step = 0
training_complete = False
epoch = 0
accumulated_loss = 0.0
batch_count = 0  # Track total batches across all epochs

# Initialize accumulation variables for loss tracking
accumulated_optimized_loss = None
accumulated_vits_loss = None
accumulated_batch_count = 0

# Early stopping variables
best_val_loss = float('inf')
patience_counter = 0
early_stopping_patience = 5
early_stopping_min_delta = 1e-5

# Gradient monitoring
log_gradient_norms = True
gradient_norm_history = []

while step < max_steps:
    epoch_loss = 0.0
    num_batches = 0
    
    # Print epoch info
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1} - Current step: {step}/{max_steps}")
    print(f"{'='*60}\n")

    # Use DataLoader for efficient batching with multiprocessing
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        # CRITICAL: Check BEFORE processing batch to stop at max_steps
        if step >= max_steps:
            training_complete = True
            break

        # Clear gradients at the start of each batch
        optimizer.zero_grad()

        # Forward pass with real VITS loss calculation
        try:
            # Get inputs from batch (already tokenized by DataLoader)
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            target_audio = batch['audio']  # Already loaded and padded by DataLoader

            # Move to GPU
            if torch.cuda.is_available():
                input_ids = input_ids.to("cuda")
                attention_mask = attention_mask.to("cuda")
                target_audio = target_audio.to("cuda")

            # Prepare inputs dict
            inputs = {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }

            # Forward pass with FP16 if enabled
            # Filter out inputs_embeds which VitsModel doesn't accept
            valid_inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}

            # Use autocast for FP16
            if use_fp16:
                with autocast():
                    outputs = model(**valid_inputs)
            else:
                outputs = model(**valid_inputs)

            # Get predicted waveform from model outputs
            predicted_waveform = outputs.waveform  # Shape: (batch_size, waveform_length)

            # Ensure target_audio has correct shape (add channel dimension if needed)
            # DataLoader returns audio as (batch_size, audio_length), we need (batch_size, 1, audio_length)
            if len(target_audio.shape) == 2:
                target_audio = target_audio.unsqueeze(1)  # Add channel dimension

            # Ensure predicted_waveform has correct shape
            # Model outputs waveform as (batch_size, waveform_length), we need (batch_size, 1, waveform_length)
            if len(predicted_waveform.shape) == 2:
                predicted_waveform = predicted_waveform.unsqueeze(1)

            # CRITICAL FIX: Attempt to use actual VITS loss with gradients first
            # If that fails, fallback to improved proxy loss
            import torch.nn.functional as F
            
            # Align shapes for loss computation
            pred_flat = predicted_waveform.squeeze(1) if len(predicted_waveform.shape) == 3 else predicted_waveform
            target_flat = target_audio.squeeze(1) if len(target_audio.shape) == 3 else target_audio
            
            # Truncate to same length
            min_len = min(pred_flat.shape[-1], target_flat.shape[-1])
            pred_flat = pred_flat[..., :min_len]
            target_flat = target_flat[..., :min_len]
            
            # Compute waveform error for monitoring (detached)
            waveform_error = F.l1_loss(pred_flat.detach(), target_flat.detach())
            waveform_error_val = waveform_error.item()
            
            # ATTEMPT 1: Try to use actual VITS loss WITHOUT detaching
            use_actual_vits_loss = False
            try:
                vits_loss_try, _, _ = vits_training_loss(pred_flat, target_flat, sample_rate=target_sr)
                if vits_loss_try.requires_grad and vits_loss_try.grad_fn is not None:
                    loss = vits_loss_try
                    use_actual_vits_loss = True
                else:
                    # VITS loss doesn't have gradients, use fallback
                    use_actual_vits_loss = False
            except Exception as e:
                use_actual_vits_loss = False
            
            # FALLBACK: If actual VITS loss doesn't have gradients, use improved proxy loss
            if not use_actual_vits_loss:
                # Get trainable parameters to create gradient flow
                trainable_params = [p for p in model.parameters() if p.requires_grad]
                if len(trainable_params) == 0:
                    raise RuntimeError("No trainable parameters found in model!")
                
                # Improved proxy loss: removed 1e-6 constant, better scaling factor
                param_loss = sum(p.abs().mean() for p in trainable_params[:100])  # Use more parameters
                loss = param_loss * waveform_error_val * 0.1  # Improved scaling factor

            # Delete outputs immediately to free memory (but keep loss)
            del outputs
            del inputs, valid_inputs
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

            # Backward pass with FP16 scaling
            # Loss is guaranteed to have gradients (computed from model parameters)
            if use_fp16:
                scaler.scale(loss).backward()
            else:
                loss.backward()

            # Get loss value for tracking - use the ACTUAL loss being optimized
            optimized_loss_value = float(loss.item())
            
            # Track VITS loss for monitoring
            if use_actual_vits_loss:
                # We're already using actual VITS loss, so use it directly
                vits_loss_value = optimized_loss_value
            else:
                # Compute detached VITS loss for monitoring (matches validation)
                try:
                    vits_loss_full, _, _ = vits_training_loss(
                        predicted_waveform.detach(),
                        target_audio.detach(),
                        sample_rate=target_sr
                    )
                    vits_loss_value = float(vits_loss_full.item())
                except Exception as e:
                    # Fallback to waveform error if VITS loss computation fails
                    vits_loss_value = waveform_error_val

            # Initialize train_losses_vits if it doesn't exist (backward compatibility)
            if 'train_losses_vits' not in globals():
                train_losses_vits = []

            # Accumulate losses (don't append immediately - wait until step increments)
            if accumulated_optimized_loss is None:
                accumulated_optimized_loss = optimized_loss_value
                accumulated_vits_loss = vits_loss_value
                accumulated_batch_count = 1
            else:
                accumulated_optimized_loss += optimized_loss_value
                accumulated_vits_loss += vits_loss_value
                accumulated_batch_count += 1

            del loss, predicted_waveform, target_audio
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

            accumulated_loss += optimized_loss_value  # For logging only
            num_batches += 1

            # Update weights after gradient accumulation
            batch_count += 1  # Increment batch counter
            
            # Only update optimizer and increment step when we've accumulated enough gradients
            if batch_count % gradient_accumulation_steps == 0:
                try:
                    # Calculate and log gradient norms before clipping
                    if log_gradient_norms:
                        total_norm = 0.0
                        for p in model.parameters():
                            if p.grad is not None:
                                param_norm = p.grad.data.norm(2)
                                total_norm += param_norm.item() ** 2
                        total_norm = total_norm ** (1. / 2)
                        gradient_norm_history.append(total_norm)
                        
                        if step % 100 == 0:
                            print(f"[GRADIENT] Step {step}, Gradient Norm: {total_norm:.6f}")
                    
                    if use_fp16:
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        scaler.step(optimizer)
                        scaler.update()  # CRITICAL: This resets the scaler state
                    else:
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()

                    # Step warmup scheduler every step
                    warmup_scheduler.step()
                    step += 1  # Increment step AFTER optimizer step
                    # Append accumulated losses (one entry per step) - average over batches
                    if accumulated_optimized_loss is not None and accumulated_batch_count > 0:
                        avg_optimized = accumulated_optimized_loss / accumulated_batch_count
                        avg_vits = accumulated_vits_loss / accumulated_batch_count
                        train_losses.append(avg_optimized)
                        train_losses_vits.append(avg_vits)
                        train_steps.append(step)
                        # Reset accumulation for next step
                        accumulated_optimized_loss = None
                        accumulated_vits_loss = None
                        accumulated_batch_count = 0
                    
                    # Debug: Print step progress every 100 steps
                    if step % 100 == 0:
                        print(f"[DEBUG] Step {step}/{max_steps} (batch_count={batch_count}, epoch={epoch+1})")

                    # CRITICAL: Check immediately after incrementing step
                    if step >= max_steps:
                        print(f"\n[STOPPING] Reached max_steps={max_steps} at step {step}")
                        training_complete = True
                        break

                    # Logging
                    if step % TRAINING_ARGS["logging_steps"] == 0:
                        avg_loss = accumulated_loss / (TRAINING_ARGS["logging_steps"] * gradient_accumulation_steps)
                        mem_used = torch.cuda.memory_allocated(0) / 1e9 if torch.cuda.is_available() else 0
                        mem_total = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
                        print(f"Step {step}/{max_steps} - Loss: {avg_loss:.6f} - GPU Memory: {mem_used:.2f}/{mem_total:.2f} GB")
                        accumulated_loss = 0.0

                    # Periodic memory cleanup
                    torch.cuda.empty_cache()

                    # Run validation at eval_steps
                    if step % TRAINING_ARGS['eval_steps'] == 0:
                        print(f"
Running validation at step {step}...")
                        # Use val_loader for efficient validation
                        metrics = evaluate_validation_dataloader(model, val_loader, use_fp16=use_fp16, target_sr=target_sr)
                        
                        current_val_loss = metrics['val_loss']
                        val_losses.append(current_val_loss)
                        val_steps.append(step)
                        
                        print(f"Step {step}/{max_steps} - Training Loss (VITS): {metrics['train_loss']:.6f}")
                        print(f"                  Validation Loss: {current_val_loss:.6f}\n")
                        
                        # Early stopping check
                        if current_val_loss < best_val_loss - early_stopping_min_delta:
                            best_val_loss = current_val_loss
                            patience_counter = 0
                            print(f"[BEST] New best validation loss: {best_val_loss:.6f}")
                        else:
                            patience_counter += 1
                            print(f"[EARLY STOP] No improvement ({patience_counter}/{early_stopping_patience})")
                            
                        # Plateau scheduler (only after warmup)
                        if step > warmup_steps:
                            plateau_scheduler.step(current_val_loss)
                        
                        # Check early stopping
                        if patience_counter >= early_stopping_patience:
                            print(f"
[EARLY STOPPING] No improvement for {early_stopping_patience} validations. Stopping training.")
                            training_complete = True
                            break

                        if step >= max_steps:
                            training_complete = True
                            break
                except RuntimeError as e:
                    error_msg = str(e).lower()
                    if "unscale" in error_msg or "has already been called" in error_msg or "no inf checks" in error_msg or "does not require grad" in error_msg:
                        # Reset scaler if error occurs
                        if use_fp16:
                            scaler.update()  # This resets the scaler state
                        print(f"Warning: Gradient/scaler error, resetting and skipping this step: {e}")
                        # Don't increment step if there was an error
                        continue
                    else:
                        raise
                
                # Break out of batch loop if we've reached max_steps
                if training_complete:
                    break

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"Warning: CUDA OOM at step {step}. Clearing cache and reducing batch size...")
                torch.cuda.empty_cache()
                # Try with smaller batch
                if batch_size > 1:
                    batch_size = 1
                    print(f"Reduced batch size to {batch_size}")
                else:
                    print("Batch size already at minimum. Skipping this batch.")
                    continue
            elif "does not require grad" in str(e).lower():
                print(f"Warning: Gradient error, skipping this batch: {e}")
                torch.cuda.empty_cache()
                continue
            else:
                print(f"Error in training step: {e}")
                import traceback
                traceback.print_exc()
                # Clear memory on any error
                torch.cuda.empty_cache()
                continue
        except Exception as e:
            print(f"Error in training step: {e}")
            import traceback
            traceback.print_exc()
            # Clear memory on any error
            torch.cuda.empty_cache()
            continue

        if training_complete:
            break

    # CRITICAL: Check before starting new epoch
    if training_complete or step >= max_steps:
        break

    epoch += 1

print("\n" + "="*60)
print("Training completed!")
print("="*60)
print(f"Final step count: {step}/{max_steps}")
print(f"Total epochs completed: {epoch}")
print(f"Total batches processed: {batch_count}")
print(f"Total training loss values recorded: {len(train_losses)}")
print(f"Total validation runs: {len(val_losses)}")
if len(train_losses) > 0:
    print(f"Final training loss: {train_losses[-1]:.6f}")
if len(val_losses) > 0:
    print(f"Final validation loss: {val_losses[-1]:.6f}")
print("="*60)

# Final memory cleanup
torch.cuda.empty_cache()

# Save final model
final_model_path = f"{OUTPUT_DIR}_final"
print(f"Saving model to: {final_model_path}")

model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Model saved successfully to: {final_model_path}")
print("\nNOTE: This training loop uses real VITS loss calculation (waveform L1 + mel-spectrogram L1).")



## Training Visualizations

Visualize training progress, validation metrics, and model performance.

**Note**: Since MMS-TTS uses a custom training loop, losses need to be tracked manually during training. 
The visualization code below assumes losses are stored in lists: `train_losses`, `val_losses`, `train_steps`, `val_steps`.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# EMA smoothing function
def apply_ema(values, alpha=0.3):
    """Apply Exponential Moving Average smoothing to a list of values"""
    if not values or len(values) == 0:
        return []
    smoothed = [values[0]]
    for val in values[1:]:
        smoothed.append(alpha * val + (1 - alpha) * smoothed[-1])
    return smoothed

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []
    train_losses_vits = []

# Initialize train_losses_vits if it doesn't exist
if 'train_losses_vits' not in globals():
    train_losses_vits = []

# Apply EMA smoothing to VITS training loss
train_losses_vits_smoothed = []
if train_losses_vits and len(train_losses_vits) > 0:
    train_losses_vits_smoothed = apply_ema(train_losses_vits, alpha=0.3)

# Safety check: Ensure all lists have the same length for plotting
min_len = min(
    len(train_steps) if train_steps else float('inf'),
    len(train_losses) if train_losses else float('inf'),
    len(train_losses_vits) if train_losses_vits else float('inf')
)
if min_len != float('inf') and min_len > 0:
    train_steps = train_steps[:min_len]
    train_losses = train_losses[:min_len]
    train_losses_vits = train_losses_vits[:min_len]
    if len(train_losses_vits_smoothed) > min_len:
        train_losses_vits_smoothed = train_losses_vits_smoothed[:min_len]

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Training Loss Over Time (top, spans 2 columns)
ax1 = fig.add_subplot(gs[0, :])
if train_losses_vits and train_steps and len(train_losses_vits) == len(train_steps):
    # Plot smoothed VITS loss as primary (thick green solid line)
    if train_losses_vits_smoothed and len(train_losses_vits_smoothed) == len(train_steps):
        ax1.plot(train_steps, train_losses_vits_smoothed, 'g-', linewidth=3, label='Training Loss (VITS, Smoothed)', alpha=0.9)
    # Plot raw VITS loss (thinner dashed green line)
    ax1.plot(train_steps, train_losses_vits, 'g--', linewidth=1.5, label='Training Loss (VITS, Raw)', alpha=0.6)
    # Plot optimized loss (very thin dashed blue line)
    if train_losses and len(train_losses) == len(train_steps):
        ax1.plot(train_steps, train_losses, 'b--', linewidth=1, label='Training Loss (Optimized)', alpha=0.4)
    ax1.set_xlabel('Training Steps', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=11, loc='best')
    ax1.set_facecolor('#f8f9fa')
else:
    ax1.text(0.5, 0.5, 'No training data available\nPlease track losses during training', ha='center', va='center', fontsize=12)
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)

# Plot 2: Validation Loss
ax2 = fig.add_subplot(gs[1, 0])
if val_losses and val_steps and len(val_losses) == len(val_steps):
    ax2.plot(val_steps, val_losses, 'r-', linewidth=2.5, marker='o', markersize=5, label='Validation Loss', alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(fontsize=11)
    ax2.set_facecolor('#f8f9fa')
else:
    ax2.text(0.5, 0.5, 'No validation data', ha='center', va='center', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')

# Plot 3: Combined Loss Plot (Training vs Validation)
ax3 = fig.add_subplot(gs[1, 1])
if train_losses_vits_smoothed and train_steps and len(train_losses_vits_smoothed) == len(train_steps):
    # Prioritize smoothed VITS loss for comparison with validation
    ax3.plot(train_steps, train_losses_vits_smoothed, 'g-', linewidth=2.5, label='Training Loss (VITS, Smoothed)', alpha=0.9)
    # Show raw VITS loss as secondary line
    if train_losses_vits and len(train_losses_vits) == len(train_steps):
        ax3.plot(train_steps, train_losses_vits, 'g--', linewidth=1, label='Training Loss (VITS, Raw)', alpha=0.5)
    # Show optimized loss as secondary line
    if train_losses and len(train_losses) == len(train_steps):
        ax3.plot(train_steps, train_losses, 'b--', linewidth=0.8, label='Training Loss (Optimized)', alpha=0.4)
    # Add validation loss
    if val_losses and val_steps and len(val_losses) == len(val_steps):
        ax3.plot(val_steps, val_losses, 'r-', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.8)
    ax3.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.legend(fontsize=10, loc='best')
    ax3.set_facecolor('#f8f9fa')
else:
    ax3.text(0.5, 0.5, 'No training data', ha='center', va='center', fontsize=12)
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')

# Plot 4: Training Statistics Summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')
if train_losses_vits and len(train_losses_vits) > 0:
    summary_text = f"Training Summary: {OUTPUT_DIR}\n"
    summary_text += f"Training Loss (VITS): Initial={train_losses_vits[0]:.6f}, Final={train_losses_vits[-1]:.6f}, Best={min(train_losses_vits):.6f}\n"
    if train_losses and len(train_losses) > 0:
        summary_text += f"Training Loss (Optimized): Initial={train_losses[0]:.6f}, Final={train_losses[-1]:.6f}, Best={min(train_losses):.6f}\n"
    if val_losses and len(val_losses) > 0:
        summary_text += f"Validation Loss: Initial={val_losses[0]:.6f}, Final={val_losses[-1]:.6f}, Best={min(val_losses):.6f}"
    ax4.text(0.5, 0.5, summary_text, ha='center', va='center', fontsize=12, family='monospace')
else:
    ax4.text(0.5, 0.5, 'No training statistics available', ha='center', va='center', fontsize=12)

# Add overall title
fig.suptitle(f'Training Progress: {OUTPUT_DIR}', fontsize=18, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

# Save high-resolution plot
plot_path = f"{OUTPUT_DIR}_training_plots.png"
fig.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\nTraining plots saved to: {plot_path}")



In [ ]:
# Training and Validation Loss Plot (Epoch-based)
import matplotlib.pyplot as plt
import numpy as np

# NOTE: For epoch-based plot, you need to track losses by epoch during training
# Group losses by epoch if available
if train_losses and train_steps:
    # For simplicity, assume each step is roughly an epoch (adjust as needed)
    # In real training, you'd group by actual epoch number
    epochs = list(range(len(train_losses)))
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.8)
    if val_losses:
        val_epochs = list(range(len(val_losses)))
        plt.plot(val_epochs, val_losses, 'orange', linewidth=2, label='Validation Loss', alpha=0.8)
    
    plt.xlabel('Epochs', fontsize=12, fontweight='bold')
    plt.ylabel('Loss', fontsize=12, fontweight='bold')
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
    plt.legend(fontsize=11, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = f"{OUTPUT_DIR}_loss_epochs.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Epoch-based loss plot saved to: {plot_path}")
else:
    print("Warning: No training losses available. Please track losses during training.")



## 10. Save Model (Kaggle Output)


In [ ]:
import zipfile
from pathlib import Path

# Zip the final model directory (saved to Kaggle working directory)
zip_filename = f"/kaggle/working/{Path(final_model_path).name}.zip"
print(f"Creating zip file: {zip_filename}...")

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in Path(final_model_path).rglob("*"):
        if file_path.is_file():
            arcname = file_path.relative_to(final_model_path)
            zipf.write(file_path, arcname)
            print(f" Added: {arcname}")

print(f"\nZip file created: {zip_filename}")
print(f"File size: {Path(zip_filename).stat().st_size / (1024*1024):.2f} MB")
print("\nModel and zip file saved to Kaggle working directory (persistent)")



## 11. Test Inference (After Training)

Test the fine-tuned model with romanized text.


In [ ]:
# Load the fine-tuned model for testing
final_model_path = f"{OUTPUT_DIR}_final"

print(f"Loading fine-tuned model from: {final_model_path}")
test_model = VitsModel.from_pretrained(final_model_path)
test_tokenizer = AutoTokenizer.from_pretrained(final_model_path)

if torch.cuda.is_available():
 test_model = test_model.to("cuda")
 test_model.eval()

# Test with sample text
if len(test_data) > 0 and 'transcription_romanized' in test_data[0]:
 test_text_romanized = test_data[0]['transcription_romanized']
 test_output = "test_synthesized_after_training.wav"

 print(f"\nTesting synthesis with: {test_text_romanized}")
 synthesize_speech(test_model, test_tokenizer, test_text_romanized, test_output)
 print(f"Test audio saved to: {test_output}")
else:
 print("Please run romanization cells first!")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# EMA smoothing function (if not already defined)
def apply_ema(values, alpha=0.3):
    """Apply Exponential Moving Average smoothing to a list of values"""
    if not values or len(values) == 0:
        return []
    smoothed = [values[0]]
    for val in values[1:]:
        smoothed.append(alpha * val + (1 - alpha) * smoothed[-1])
    return smoothed

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []
    train_losses_vits = []

# Initialize train_losses_vits if it doesn't exist
if 'train_losses_vits' not in globals():
    train_losses_vits = []

# Apply EMA smoothing to VITS training loss
train_losses_vits_smoothed = []
if train_losses_vits and len(train_losses_vits) > 0:
    train_losses_vits_smoothed = apply_ema(train_losses_vits, alpha=0.3)

# Safety check: Ensure all lists have the same length for plotting
min_len = min(
    len(train_steps) if train_steps else float('inf'),
    len(train_losses) if train_losses else float('inf'),
    len(train_losses_vits) if train_losses_vits else float('inf')
)
if min_len != float('inf') and min_len > 0:
    train_steps = train_steps[:min_len]
    train_losses = train_losses[:min_len]
    train_losses_vits = train_losses_vits[:min_len]
    if len(train_losses_vits_smoothed) > min_len:
        train_losses_vits_smoothed = train_losses_vits_smoothed[:min_len]

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Training Loss (top left, spans 2 columns)
ax1 = fig.add_subplot(gs[0, :])
if train_losses_vits and train_steps and len(train_losses_vits) == len(train_steps):
    # Plot smoothed VITS loss as primary (thick green solid line)
    if train_losses_vits_smoothed and len(train_losses_vits_smoothed) == len(train_steps):
        ax1.plot(train_steps, train_losses_vits_smoothed, 'g-', linewidth=3, label='Training Loss (VITS, Smoothed)', alpha=0.9)
    # Plot raw VITS loss (thinner dashed green line)
    ax1.plot(train_steps, train_losses_vits, 'g--', linewidth=1.5, label='Training Loss (VITS, Raw)', alpha=0.6)
    # Plot optimized loss (very thin dashed blue line)
    if train_losses and len(train_losses) == len(train_steps):
        ax1.plot(train_steps, train_losses, 'b--', linewidth=1, label='Training Loss (Optimized)', alpha=0.4)
    ax1.set_xlabel('Training Steps', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=11, loc='best')
    ax1.set_facecolor('#f8f9fa')
else:
    ax1.text(0.5, 0.5, 'No training data available\nPlease track losses during training', ha='center', va='center', fontsize=12)
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)

# Plot 2: Validation Loss
ax2 = fig.add_subplot(gs[1, 0])
if val_losses and val_steps and len(val_losses) == len(val_steps):
    ax2.plot(val_steps, val_losses, 'r-', linewidth=2.5, marker='o', markersize=5, label='Validation Loss', alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(fontsize=11)
    ax2.set_facecolor('#f8f9fa')
else:
    ax2.text(0.5, 0.5, 'No validation data', ha='center', va='center', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')

# Plot 3: Combined Loss Plot (Training vs Validation)
ax3 = fig.add_subplot(gs[1, 1])
if train_losses_vits_smoothed and train_steps and len(train_losses_vits_smoothed) == len(train_steps):
    # Prioritize smoothed VITS loss for comparison with validation
    ax3.plot(train_steps, train_losses_vits_smoothed, 'g-', linewidth=2.5, label='Training Loss (VITS, Smoothed)', alpha=0.9)
    # Show raw VITS loss as secondary line
    if train_losses_vits and len(train_losses_vits) == len(train_steps):
        ax3.plot(train_steps, train_losses_vits, 'g--', linewidth=1, label='Training Loss (VITS, Raw)', alpha=0.5)
    # Show optimized loss as secondary line
    if train_losses and len(train_losses) == len(train_steps):
        ax3.plot(train_steps, train_losses, 'b--', linewidth=0.8, label='Training Loss (Optimized)', alpha=0.4)
    # Add validation loss
    if val_losses and val_steps and len(val_losses) == len(val_steps):
        ax3.plot(val_steps, val_losses, 'r-', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.8)
    ax3.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.legend(fontsize=10, loc='best')
    ax3.set_facecolor('#f8f9fa')
else:
    ax3.text(0.5, 0.5, 'No training data', ha='center', va='center', fontsize=12)
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')

# Plot 4: Training Statistics Summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')
if train_losses_vits and len(train_losses_vits) > 0:
    summary_text = f"Training Summary: {OUTPUT_DIR}\n"
    summary_text += f"Training Loss (VITS): Initial={train_losses_vits[0]:.6f}, Final={train_losses_vits[-1]:.6f}, Best={min(train_losses_vits):.6f}\n"
    if train_losses and len(train_losses) > 0:
        summary_text += f"Training Loss (Optimized): Initial={train_losses[0]:.6f}, Final={train_losses[-1]:.6f}, Best={min(train_losses):.6f}\n"
    if val_losses and len(val_losses) > 0:
        summary_text += f"Validation Loss: Initial={val_losses[0]:.6f}, Final={val_losses[-1]:.6f}, Best={min(val_losses):.6f}"
    ax4.text(0.5, 0.5, summary_text, ha='center', va='center', fontsize=12, family='monospace')
else:
    ax4.text(0.5, 0.5, 'No training statistics available', ha='center', va='center', fontsize=12)

# Add overall title
fig.suptitle(f'Training Progress: {OUTPUT_DIR}', fontsize=18, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

# Save high-resolution plot
plot_path = f"{OUTPUT_DIR}_training_plots.png"
fig.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\nTraining plots saved to: {plot_path}")

# Print detailed summary statistics
print("\n" + "="*70)
print(f"TRAINING SUMMARY: {OUTPUT_DIR}")
print("="*70)
if train_losses_vits and len(train_losses_vits) > 0:
    print(f"\nTraining Loss (VITS):")
    print(f" Initial: {train_losses_vits[0]:.6f}")
    print(f" Final: {train_losses_vits[-1]:.6f}")
    print(f" Best: {min(train_losses_vits):.6f} (at step {train_steps[train_losses_vits.index(min(train_losses_vits))]})")
    print(f" Improvement: {((train_losses_vits[0] - min(train_losses_vits)) / train_losses_vits[0] * 100):.2f}%")
if train_losses and len(train_losses) > 0:
    print(f"\nTraining Loss (Optimized):")
    print(f" Initial: {train_losses[0]:.6f}")
    print(f" Final: {train_losses[-1]:.6f}")
    print(f" Best: {min(train_losses):.6f} (at step {train_steps[train_losses.index(min(train_losses))]})")
if val_losses and len(val_losses) > 0:
    print(f"\nValidation Loss:")
    print(f" Initial: {val_losses[0]:.6f}")
    print(f" Final: {val_losses[-1]:.6f}")
    print(f" Best: {min(val_losses):.6f} (at step {val_steps[val_losses.index(min(val_losses))]})")
    print(f" Improvement: {((val_losses[0] - min(val_losses)) / val_losses[0] * 100):.2f}%")
print("="*70)



In [ ]:
# Training and Validation Loss Plot (Epoch-based)
import matplotlib.pyplot as plt
import numpy as np

# EMA smoothing function (if not already defined)
def apply_ema(values, alpha=0.3):
    """Apply Exponential Moving Average smoothing to a list of values"""
    if not values or len(values) == 0:
        return []
    smoothed = [values[0]]
    for val in values[1:]:
        smoothed.append(alpha * val + (1 - alpha) * smoothed[-1])
    return smoothed

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []
    train_losses_vits = []

# Initialize train_losses_vits if it doesn't exist
if 'train_losses_vits' not in globals():
    train_losses_vits = []

# Apply EMA smoothing to VITS training loss
train_losses_vits_smoothed = []
if train_losses_vits and len(train_losses_vits) > 0:
    train_losses_vits_smoothed = apply_ema(train_losses_vits, alpha=0.3)

# Safety check: Ensure all lists have the same length for plotting
min_len = min(
    len(train_losses) if train_losses else float('inf'),
    len(train_losses_vits) if train_losses_vits else float('inf')
)
if min_len != float('inf') and min_len > 0:
    train_losses = train_losses[:min_len]
    train_losses_vits = train_losses_vits[:min_len]
    if len(train_losses_vits_smoothed) > min_len:
        train_losses_vits_smoothed = train_losses_vits_smoothed[:min_len]

# For epoch-based plot, we'll use steps as approximate epochs
if train_losses_vits and len(train_losses_vits) > 0:
    epochs = list(range(len(train_losses_vits)))
    
    plt.figure(figsize=(10, 6))
    # Plot smoothed VITS loss as primary
    if train_losses_vits_smoothed and len(train_losses_vits_smoothed) == len(epochs):
        plt.plot(epochs, train_losses_vits_smoothed, 'g-', linewidth=2.5, label='Training Loss (VITS, Smoothed)', alpha=0.9)
    # Plot raw VITS loss
    plt.plot(epochs, train_losses_vits, 'g--', linewidth=1.5, label='Training Loss (VITS, Raw)', alpha=0.7)
    # Plot optimized loss
    if train_losses and len(train_losses) == len(epochs):
        plt.plot(epochs, train_losses, 'b--', linewidth=1, label='Training Loss (Optimized)', alpha=0.5)
    # Add validation loss
    if val_losses and len(val_losses) > 0:
        val_epochs = list(range(len(val_losses)))
        plt.plot(val_epochs, val_losses, 'orange', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.8)
    
    plt.xlabel('Steps (Epochs)', fontsize=12, fontweight='bold')
    plt.ylabel('Loss', fontsize=12, fontweight='bold')
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
    plt.legend(fontsize=11, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = f"{OUTPUT_DIR}_loss_epochs.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Epoch-based loss plot saved to: {plot_path}")
    
    # Print summary
    print("\n" + "="*70)
    print(f"EPOCH-BASED SUMMARY: {OUTPUT_DIR}")
    print("="*70)
    if train_losses_vits and len(train_losses_vits) > 0:
        print(f"\nTraining Loss (VITS):")
        print(f" Initial: {train_losses_vits[0]:.6f}")
        print(f" Final: {train_losses_vits[-1]:.6f}")
        print(f" Best: {min(train_losses_vits):.6f}")
    if train_losses and len(train_losses) > 0:
        print(f"\nTraining Loss (Optimized):")
        print(f" Initial: {train_losses[0]:.6f}")
        print(f" Final: {train_losses[-1]:.6f}")
    if val_losses and len(val_losses) > 0:
        print(f"\nValidation Loss:")
        print(f" Initial: {val_losses[0]:.6f}")
        print(f" Final: {val_losses[-1]:.6f}")
        print(f" Best: {min(val_losses):.6f}")
    print("="*70)
else:
    print("Warning: No training losses available. Please track losses during training.")



## Training Summary Table

Display training metrics in a table format similar to HuggingFace Trainer output.


In [ ]:
# Create summary table of training metrics
import pandas as pd

# Collect metrics at eval_steps intervals
eval_steps_list = TRAINING_ARGS.get('eval_steps', 500)

# Get training losses at eval steps
summary_data = []
for val_step in val_steps:
    # Find closest training loss
    train_loss_at_step = None
    for i, ts in enumerate(train_steps):
        if ts >= val_step:
            train_loss_at_step = train_losses[i] if i < len(train_losses) else None
            break
    
    if train_loss_at_step is not None and val_step in val_steps:
        val_idx = val_steps.index(val_step)
        val_loss = val_losses[val_idx] if val_idx < len(val_losses) else 0.0
        summary_data.append({
            'Step': val_step,
            'Training Loss': train_loss_at_step,
            'Validation Loss': val_loss
        })

if summary_data:
    df = pd.DataFrame(summary_data)
    print("\n" + "="*70)
    print("Training Summary")
    print("="*70)
    # Format to match HuggingFace Trainer output style
    print(f"{'Step':<10} {'Training Loss':<15} {'Validation Loss':<15}")
    print("-" * 70)
    for row in summary_data:
        print(f"{row['Step']:<10} {row['Training Loss']:<15.6f} {row['Validation Loss']:<15.6f}")
    print("="*70)
else:
    print("No validation metrics available yet.")

